# Class 3 - Usar ficheros desde S3

## Objetivo

Usar datos reales de gran tamaño almacenados en S3

### Steps

1. subir datos a S3
2. utilizar S3 como fuente de datos de Tweets 
3. contestar preguntas sobre estos datos

# Parte 1: Operaciones en S3

Para praticar las operaciones disponibles en S3 usaremos la consola del Learner Lab. 

## Buckets 

### Crear un bucket 

Para crear el bucket `bucket-de-ejemplo`, el comando es: 

```aws
aws s3api create-bucket -bucket bucket-de-ejemplo
```

Con el comando 
```aws
aws s3api list-buckets  
```

podemos comprobar que el bucket haya sido creado. 

&#x26A0; **Importante!** No pueden existir globalmente en S3 dos buckets con el mismo nombre, aunque sean creados por personas/cuentas diferentes. 

### Eliminar un bucket 

Para eliminar el bucket `bucket-de-ejemplo`, el comando es: 

```aws
aws s3api delete-bucket -bucket bucket-de-ejemplo
```

Con el comando 
```aws
aws s3api list-buckets  
```

podemos comprobar que el bucket haya sido eliminado. 

## Ficheros

### Hacer el listado de ficheros en un directorio 

En S3 los directorios como si no existen, pero cuando dos objetos tienen un prefijo comun separado por `/` este prefijo comun se puede interpretar como un directorio. Por ejemplo, el fichero con key `test/sub/file1` y el fichero con key `test/sub/file2` comparten el mismo prefijo `test/sub`, que podemos interpretar como que en el bucket contiene un directorio `test` que contiene un directorio `sub`, que contiene dos ficheros `file1` y `file2`. 

Por lo tanto, imaginando que estos ficheros sean dentro de un bucket llamado `my-bucket`, podemos pedir por consola de ver el contenido del directorio `test/sub` con el siguiente comando:

```
aws s3 ls s3://my-bucket/test/sub 
```

### Copiar un fichero a S3 

Un fichero `my-file.txt` desde el ordenador se puede copiar a S3 con el siguiente comando:

```
aws s3 cp my-file.txt s3://my-bucket/my-dir/my-file.txt
```

También es posible copiar un fichero de S3 a S3, siempre y cuando tengamos los permisos suficientes para realizar la operación (permisos de lectura del fichero de origen, y permiso de escritura al bucket de destino).

Por ejemplo el siguiente comando: 
```
aws s3 cp s3://my-bucket/file1.txt s3://my-other-bucket/copy/file1
```

copia el fichero `file1.txt` desde el bucket `my-bucket` al bucket `my-other-bucket` con el key `copy/file1`. Esto quiere decir que el fichero `file1` estará automáticamente en un directorio `copy` que se creará sin necesidad de existir previamente. Con el comando 

```
aws s3 ls s3://my-other-bucket/copy/ 
```
confirmaremos que el fichero está en el directorio `copy` bajo el nombre `file1` 

### Otras operaciones sobre ficheros "locales"

Para hacer pruebas, puede ser útil crear ficheros **desde la consola del Learner Lab**. 

El siguiente comando crea un fichero vacío llamado `my-file.txt`

```bash
touch my-file.txt
```

El siguiente comando imprime en pantalla el listado de ficheros en el directorio actual: 

```bash
ls
```

También podemos crear un fichero con un contenido desde la consola: 

```bash
echo '12345' > my-second-file.txt 
```

El contenido del fichero `my-second-file.txt` será el texto `12345`

## Actividad 1 (en clase)
**Usando la consola del Learner Lab**, ejecuta lo siguiente:

- Crea un bucket llamado `mudab-ixxxxx` donde `ixxxxx` es tu número identificativo de estudiante
- Crea un fichero local con el contenido `abcde` y nombre `file1.txt`
- Copia el fichero en tu bucket, bajo el nombre `test/consola/file1`
- Copia de nuevo el fichero, bajo el nombre `test/consola/file2`
- Comprueba el contenido de tu bucket bajo la key `test/consola/`, deberían aparecer 2 ficheros
- Borra el fichero `file2`
- Comprueba el contenido de tu bucket bajo la key `test/consola/`, debería aparecer 1 fichero

### Descarga en tu ordenador  
Si quieres tener los ficheros en tu ordenador para hacer pruebas por tu cuenta, pues los objetos públicos de S3 tiene un URL desde donde descargar el objeto. La versión comprimida de los tweet está disponible aquí: https://mudab-2025-big-data.s3.us-east-1.amazonaws.com/twitter-data-compressed/twitter-data-from-eurovision-2018-splits.zip

# Parte 2: S3 de manera programática 

En esta parte, añadiremos el acceso a S3 como sistema de ficheros a través de una librería llamada [s3fs](https://s3fs.readthedocs.io/en/latest/). Esta librería nos permite acceder a S3 como si fuera un sistema de ficheros local. 

Aunque el acceso a S3 de manera programatica requiera credenciales, la ejecución en entorno SageMaker nos permite omitir este paso de configuración, y tener un código más sencillo. 



In [ ]:
import s3fs
s3 = s3fs.S3FileSystem(anon=False)

bucket='mudab-2025-big-data'
data_key = 'twitter-data/Eurovision-00.json'
data_location = 's3://{}/{}'.format(bucket, data_key)

### Operaciones sobre ficheros con la librería s3fs 

La librería s3fs es un ejemplo de acceso programático a los servicios de AWS. En este caso, nos permite acceder a S3 y hacer operaciones utilizando Python. Las operaciones sobre ficheros explicadas arriba se traducen en pequeños *snippets* de código.

#### Hacer el listado de ficheros en un directorio

```python
s3.ls("my-bucket/test/sub")
```

#### Copiar un fichero a S3

```python
s3.put("my-file.txt", "s3://my-bucket/my-dir/my-file.txt")
```

#### Borrar un fichero desde S3 

```python
s3.rm("my-bucket/my-dir/my-file.txt")
```

## Actividad 2 (en clase)

Primero, crea en tu ordenador un fichero local con el contenido que quieras, y nombre `f1.txt`, luego cargalo a tu entorno Jupyter a través del icono de upload &#x2B06;. Usaremos el mismo bucket creado en la actividad anterior

Luego, **Usando el entorno de Jupyter**, ejecuta lo siguiente:

- Copia el fichero en tu bucket, bajo el nombre `test/jupyter/file1`
- Copia de nuevo el fichero, bajo el nombre `test/jupyter/file2`
- Comprueba el contenido de tu bucket bajo la key `test/jupyter/`, deberían aparecer 2 ficheros
- Borra el fichero `file2`
- Comprueba el contenido de tu bucket bajo la key `test/jupyter/`, debería aparecer 1 fichero

# Parte 3: Big data desde S3 

En esta parte, intentaremos procesar la colección entera de Tweets que reside en S3. Los pasos a seguir serán los siguientes: 

1. Desde la consola del Learner Lab, copiar la colección entera desde el bucket de origen (`mudab-2025-big-data`) bajo el path `twitter-data`. Los ficheros se denominan `Eurovision-XX.json` donde `XX` es un número entre `00` y `09`. Para hacer una copia recursiva de ficheros se puede utilizar el comando de copia de una manera más sofisticada utilizando la opción `--recursive`. 
```bash
aws s3 cp —recursive s3://mudab-2023/twitter-data/ s3://mudab2023-i123456/input/
```

2. Utilizar el mísmo código de la versión anterior para contestar a las siguientes preguntas: 
- Cuántos Tweeets son en español? 
- Cual son las 100 palabras más frecuentes en español? 

## Modelo de Tweets

Similar a la clase anterior, creamos un modelo de Tweet 

In [ ]:
from dataclasses import dataclass

@dataclass
class Tweet:
  """Class to model a Tweet"""
  id: int         # The unique ID of a tweet
  content: str    # The textual content of a tweet
  author: str     # The nickname of the author of the tweet
  language: str   # The language of the tweet


## Procesamiento de input (ETL) 

Similar a la clase anterior, almacenamos todos los tweets en memoria. 

**Pregunta**: funcionarà esto para nuestro input completo? Que hacer si así no es? 

In [ ]:
import json, dataclasses

tweets = []

def parse_line(line: str):
  """Try to parse a string into a Person"""
  try:
    parsed = json.loads(line)
    return Tweet(parsed['id'], parsed['text'], parsed['user']['name'], parsed['lang'])
  except Exception as e:
    print(f"Error parsing '{line}': {e}")

with s3.open(data_location) as input:
  for line in input.readlines():
    if len(line.strip()) > 0:
      tweet = parse_line(line)
      if tweet: # We add only if the tweet is not 'None'
         tweets.append(tweet)

for modeled_tweet in tweets[0:10]:
  print(modeled_tweet)

# Procesamiento de datos

In [ ]:
import json, dataclasses

def read_clean_tweets(input: str):
  tweets = []
  with open(input, 'r') as f:
    lines = f.readlines()
  for line in lines:
    parsed = json.loads(line)
    tweet = Tweet(**parsed)
    tweets.append(tweet)
  return tweets

def count_tweets(language: str, tweets: list[Tweet]):
  count = 0
  for tweet in tweets:
    if (tweet.language == language):
      count = count + 1
  return count

def most_frequent_word(tweets: list[Tweet]):
  count = {}
  for tweet in tweets:
    words = tweet.content.split(' ')
    for word in words:
      if (word in count):
        new_val = count[word] + 1
        count[word] = new_val
      else:
        count[word] = 1
  return dict(sorted(count.items(), key=lambda item: item[1], reverse=True))

spanish_tweets_count = count_tweets('es', tweets)

words_by_frequence = list(most_frequent_word(tweets).items())[0:100]

print(spanish_tweets_count)
print(words_by_frequence)

# Question 3 (Home, Moodle)

Los ejemplos de arriba se han hecho con *un solo* fichero, pero las siguientes preguntas aplican a **TODA** la colección (el conjunto de 10 ficheros). 

- 3.1. Cuántos tweets en español son originales (es decir, no son retweets)? 
- 3.2. Cual es el porcentaje de tweets para cada lenguaje? Es decir, de toda la colleccion, XX% son en idioma YY, ZZ% son en idioma WW, etc...
- 3.3. Cuáles son las palabras más frecuentse en castellano? 

Añade tu respuesta en un bloque de Jupyter aquí abajo 